In [3]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import sys
sys.path.append('D:/Projects/Crypto/Scripts/API')
import CoinsInfo as ce

In [10]:
data = ce.coin_info(0)

{'name': 'Bitcoin', 'symbol': '₿', 'rank': 1, 'age': 5624, 'color': '#fa9e32', 'png32': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/32/btc.png', 'png64': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/64/btc.png', 'webp32': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/32/btc.webp', 'webp64': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/64/btc.webp', 'exchanges': 195, 'markets': 2978, 'pairs': 893, 'categories': [], 'allTimeHighUSD': 73781.24185982272, 'circulatingSupply': 19704318, 'totalSupply': 19704318, 'maxSupply': 21000000, 'links': {'website': 'https://bitcoin.org', 'whitepaper': 'https://bitcoin.org/bitcoin.pdf', 'twitter': None, 'reddit': 'https://reddit.com/r/bitcoin', 'telegram': None, 'discord': None, 'medium': None, 'instagram': None, 'tiktok': None, 'youtube': None, 'linkedin': None, 'twitch': None, 'spotify': None, 'naver': None, 'wechat': None, 'soundcloud': None}, 'rate': 68602.088603

In [7]:
while True:
    sleep(20)
    data = ce.coin_info(0)
    producer = KafkaProducer(bootstrap_servers=['52.14.28.124:9092'],
                            value_serializer=lambda x: dumps(x).encode('utf-8'))

    producer.send('crypto', value=data)

{'name': 'Bitcoin', 'symbol': '₿', 'rank': 1, 'age': 5624, 'color': '#fa9e32', 'png32': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/32/btc.png', 'png64': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/64/btc.png', 'webp32': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/32/btc.webp', 'webp64': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/64/btc.webp', 'exchanges': 195, 'markets': 2980, 'pairs': 894, 'categories': [], 'allTimeHighUSD': 73781.24185982272, 'circulatingSupply': 19704318, 'totalSupply': 19704318, 'maxSupply': 21000000, 'links': {'website': 'https://bitcoin.org', 'whitepaper': 'https://bitcoin.org/bitcoin.pdf', 'twitter': None, 'reddit': 'https://reddit.com/r/bitcoin', 'telegram': None, 'discord': None, 'medium': None, 'instagram': None, 'tiktok': None, 'youtube': None, 'linkedin': None, 'twitch': None, 'spotify': None, 'naver': None, 'wechat': None, 'soundcloud': None}, 'rate': 68407.988889